In [1]:
import numpy as np
from variational_staple.unordered import staple, vstaple

In [2]:
# We define the true performance of 4 raters tasked to classify "things"
# between three classes. Since they are not perfect raters, they
# have a probability of classifiying a "thing" as A even though it was B.
# Each performance matrix gives the probability of rater n of choosing
# class A when it should have been B:
#   true_perf[n, A, B] = P(rater [n] says A | true class is B)
true_perf = [
    np.eye(3),                                              # rater 1 (perfect)
    [[0.8, 0.1, 0.1], [0.1, 0.8, 0.1], [0.1, 0.1, 0.8]],    # rater 2 (imperfect)
    np.ones([3, 3]) / 3,                                    # rater 3 (chance)
    [[0.1, 0.8, 0.1], [0.8, 0.1, 0.1], [0.1, 0.1, 0.8]],    # rater 4 (confused)
]
true_perf = np.asarray(true_perf)

# check that columns sum to one
print('Raters\' confusion')
print(true_perf)

Raters' confusion
[[[1.         0.         0.        ]
  [0.         1.         0.        ]
  [0.         0.         1.        ]]

 [[0.8        0.1        0.1       ]
  [0.1        0.8        0.1       ]
  [0.1        0.1        0.8       ]]

 [[0.33333333 0.33333333 0.33333333]
  [0.33333333 0.33333333 0.33333333]
  [0.33333333 0.33333333 0.33333333]]

 [[0.1        0.8        0.1       ]
  [0.8        0.1        0.1       ]
  [0.1        0.1        0.8       ]]]


In [3]:
# sample true classes
true_prop = [0.7, 0.2, 0.1]
n = 10000
true_classes = np.random.multinomial(1, true_prop, n).argmax(axis=1)   # [N]

print('Sample of true classes')
print(true_classes[:10])

# sample the raters' choice using the confusion matrices
conditional_confusion = true_perf[:, :, true_classes].transpose(2, 0, 1)          # [N, R, K]
ratings = [np.random.multinomial(1, conditional_confusion[i, j]).argmax()
           for i in range(n) for j in range(len(true_perf))]
ratings = np.asarray(ratings).reshape(n, len(true_perf))

print('Sample of ratings')
print(ratings[:10].T)

Sample of true classes
[0 0 0 1 0 0 2 0 2 1]
Sample of ratings
[[0 0 0 1 0 0 2 0 2 1]
 [0 0 0 1 0 0 2 0 2 1]
 [1 0 0 0 0 1 2 1 1 0]
 [1 1 1 0 1 1 2 2 1 0]]


In [4]:
# Fit using STAPLE

perf, posterior, prior = staple(ratings)

print('Estimated confusion matrices')
print(np.around(perf, 1))

print('Estimated class proportions')
print(np.around(prior, 1))

Estimated confusion matrices
[[[1.  0.  0. ]
  [0.  1.  0. ]
  [0.  0.  1. ]]

 [[0.8 0.1 0.1]
  [0.1 0.8 0.1]
  [0.1 0.1 0.8]]

 [[0.3 0.3 0.3]
  [0.3 0.3 0.3]
  [0.3 0.3 0.3]]

 [[0.1 0.8 0.1]
  [0.8 0.1 0.1]
  [0.1 0.1 0.8]]]
Estimated class proportions
[0.7 0.2 0.1]


In [5]:
# Fit using VSTAPLE (a variant of MAP-STAPLE)

perf, posterior, prior = vstaple(ratings)

print('Estimated confusion matrices')
print(np.around(perf, 1))

print('Estimated class proportions')
print(np.around(prior, 1))

Estimated confusion matrices
[[[1.  0.  0. ]
  [0.  1.  0. ]
  [0.  0.  1. ]]

 [[0.8 0.1 0.1]
  [0.1 0.8 0.1]
  [0.1 0.1 0.8]]

 [[0.3 0.3 0.3]
  [0.3 0.3 0.3]
  [0.3 0.3 0.4]]

 [[0.1 0.8 0.1]
  [0.8 0.1 0.1]
  [0.1 0.1 0.8]]]
Estimated class proportions
[0.7 0.2 0.1]
